In [ ]:
import geemap
import ipywidgets as widgets
import numpy as np
import pandas as pd
import proplot as plot 
import matplotlib.pyplot as plt 
from ipygee import chart
from pandas.plotting import register_matplotlib_converters
import ee

In [4]:
style = {'description_width': 'initial'}

year = widgets.IntSlider(description='Select Year:', value=2013, min=2013, max=2021, style=style)
month = widgets.IntSlider(description='Select Month:', value=5, min=1, max=12, style=style)
# hbox = widgets.HBox([year, month])

In [5]:
add_layer = widgets.Button(
    description='Add Layer',
    button_style='primary',
    tooltip='Click to add layer',
    style=style
)

output1 = widgets.Output()

hbox = widgets.HBox([year, month, add_layer])
# hbox2 = widgets.HBox([add_layer])

In [6]:
map = geemap.Map(center=[23.5121, 80.3288], zoom=5)

def submit_check(b):
    
    with output1:
        output1.clear_output()
            
        start_date = str(year.value)+ '-' + str(month.value).zfill(2) + '-01'
        end_date = str(year.value)+ '-' + str(month.value).zfill(2) + '-30'
        
        print('Computing...')
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date, end_date).min()
        nir = image.select('B5')
        red = image.select('B4')
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
        evi = image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B5'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        })
        eviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
        map.addLayer(ndvi, ndviParams, 'NDVI image')
        map.addLayer(evi, eviParams, 'EVI image')
        # print(type(map))
        print('Map generated')

add_layer.on_click(submit_check)

In [7]:
output1

Output()

In [8]:
hbox

In [9]:
# hbox2

In [10]:
map

Map(center=[23.5121, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…